# 분석 정리

1. 피처 변환 : Weekly_Sales -> log
2. 피처 추가 : Date -> year, month, week, day (주말은 어차피 금요일 밖에 없음)
3. 피처 제거 : Date -> 세분화했으니 제거
4. 피처 제거 : Fuel_Price -> Weekly_Sales와 상관관계가 보이지 않음, 그러나 파생 피처를 만들어서 쓸 수도 있음.
5. 피처 인코딩 : IsHoliday -> bool을 int로 숫자화 하기
6. 피처 결측치 : Promotion 1 ~ 5 -> 프로모션이 없는 것으로 간주해서 0으로 채움.

# 베이스라인 모델 전체 프로세스

데이터 불러오기 -> 피처 엔지니어링 -> 평가지표 계산 함수 작성 -> 모델 훈련 -> 성능 검증 -> 제출

## 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np

# 데이터 불러오기
data_path = 'data/'

train = pd.read_csv(data_path + 'train.csv', index_col='id') # 학습 데이터
test = pd.read_csv(data_path + 'test.csv', index_col='id') # 테스트 데이터
submission = pd.read_csv(data_path + 'sample_submission.csv') # 제출 샘플 데이터

train.shape, test.shape, submission.shape

((6255, 12), (180, 11), (180, 2))

## 피처 엔지니어링

In [2]:
# 분석6 : 피처 결측치
train = train.fillna(0)
test = test.fillna(0)

In [3]:
# 데이터 합치기
df = pd.concat([train, test])

df.shape

(6435, 12)

In [4]:
# 분석1 : 피처 변환
df['Weekly_Sales'] = np.log1p(df['Weekly_Sales'])

In [5]:
# 분석2 : 피처 추가
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day
df['week'] = df['day'] // 7

In [6]:
# 분석3, 4 : 피처 제거
drop_features = ['Date', 'Fuel_Price']
df = df.drop(columns=drop_features)

In [7]:
# 분석5 : 피처 인코딩
df['IsHoliday'] = df['IsHoliday'].astype(int)

In [8]:
# 데이터셋 나누기
features = df.loc[df['Weekly_Sales'].notnull()]
label = features.pop('Weekly_Sales')

X_test = df[df['Weekly_Sales'].isnull()].drop(columns='Weekly_Sales')

features.shape, label.shape, X_test.shape

((6255, 13), (6255,), (180, 13))

## 모델

In [9]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression() # 모델 생성

## 성능 검증

In [10]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

kfold = KFold(n_splits=15, shuffle=True, random_state=2001) # KFold

n_iter = 0
cv_rmse = []
pred_test = np.zeros((X_test.shape[0]))

for train_index, test_index in kfold.split(features):
    n_iter += 1
    
    X_train, X_val = features.iloc[train_index], features.iloc[test_index]
    y_train, y_val = label.iloc[train_index], label.iloc[test_index]
    
    lr_model.fit(X_train, y_train) # 모델 학습
    
    pred = lr_model.predict(X_val) # 검증용 예측
    pred_test += lr_model.predict(X_test) / 15 # 제출용 예측
    
    rmse = np.round(mean_squared_error(y_val, pred, squared=False), 4) # 평가
    
    print(f'\n#{n_iter} 교차 검증 rmse : {rmse}')
    cv_rmse.append(rmse)
    
print(f'\n## 평균 검증 rmse :', np.round(np.mean(cv_rmse), 4))


#1 교차 검증 rmse : 0.5445

#2 교차 검증 rmse : 0.5422

#3 교차 검증 rmse : 0.5516

#4 교차 검증 rmse : 0.5586

#5 교차 검증 rmse : 0.5502

#6 교차 검증 rmse : 0.5262

#7 교차 검증 rmse : 0.5372

#8 교차 검증 rmse : 0.5255

#9 교차 검증 rmse : 0.5282

#10 교차 검증 rmse : 0.5311

#11 교차 검증 rmse : 0.528

#12 교차 검증 rmse : 0.5445

#13 교차 검증 rmse : 0.5415

#14 교차 검증 rmse : 0.5375

#15 교차 검증 rmse : 0.5565

## 평균 검증 rmse : 0.5402


## 제출

In [11]:
submission['Weekly_Sales'] = np.expm1(pred_test) # 지수 변환
submission.to_csv(data_path + 'submission_baseline.csv', index=False)